In [ ]:
!pip install pymorphy2
import zipfile
import os
import pandas as pd
import nltk
import re
from nltk.tokenize import word_tokenize
import pymorphy2
import string
morph = pymorphy2.MorphAnalyzer()
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.g

True

In [ ]:
with zipfile.ZipFile('corpus_reviews.zip', 'r') as zip:
    zip.extractall('corpus_reviews')

In [ ]:
def data(folder):
    doc = []
    for filename in os.listdir(folder):
        with open(os.path.join(folder, filename), 'r', encoding='utf-8') as f:
            try:
              text = f.read()
              doc.append(text)
            except UnicodeDecodeError:
              pass
    return doc

In [ ]:
senti_mu = data('corpus_reviews/lemmatized_senti_mu')

In [ ]:
df = pd.DataFrame({'content': senti_mu})

In [ ]:
df.head()

,content
0,191 приторный и пошловатый книга с слащавый по...
1,342 крохотный комикс с мила рисовка который в ...
2,265 категорически непроработать мир и до боль ...
3,270 трудно поверить что такой тонкий книга моч...
4,87 сериал на 3 голов хороший даже не смотреть ...


In [ ]:
sw = open(r'swl_optimum.txt', encoding='utf8')
swl = []  
for line in sw:
    swl.append(line.strip())
swl.append('positive')
swl.append('negative')
swl.append('быть')

In [ ]:
def clean(text):
  text = text.lower()
  tokens = nltk.word_tokenize(text)
  clean = []
  for word in tokens: 
    token = morph.parse(word)[0].normal_form
    if word not in swl:
       clean.append(word)     
  text = ' '.join(clean)
  
  return text

In [ ]:
df['content'] = df['content'].apply(clean)

In [ ]:
df

,content
0,191 приторный пошловатый книга слащавый подсюс...
1,342 крохотный комикс мила рисовка момент заста...
2,265 категорически непроработать мир боль втори...
3,270 трудно поверить тонкий книга написать исхо...
4,87 сериал голов хороший смотреть сериал сделат...
...,...
1496,498 начинаться радужно прекрасно ведьма тёмный...
1497,151 время 1997 год место событие москва остров...
1498,324 сожаление книга оказаться интересный читат...
1499,465 план выходить убийца отношение семье.по -э...


LDA:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
NUM_TOPICS = 15
N_FIRST = 20 # сколько первых слов на тему выводить
MIN_DF = 15 # минимальное число вхождений слова в документ, чтобы мы его учитывали (включали в вектор документа)
MAX_DF = 0.95 # в каком проценте документов должно присутствовать слово, чтобы оно не рассматривалось
STW_PATH = '/content/swl_optimum.txt' # путь к документу со стоп-словами

In [ ]:
count_vectorizer = CountVectorizer(min_df=MIN_DF, max_df=MAX_DF, stop_words = swl)
data_count_vectorized = count_vectorizer.fit_transform(df["content"])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['авт', 'адм', 'ак', 'акад', 'богу', 'бу', 'буль', 'вашему', 'вв', 'вкз', 'вост', 'всякому', 'гг', 'го', 'гор', 'гос', 'гр', 'деп', 'дисс', 'дол', 'долл', 'другому', 'европ', 'ежедн', 'ей', 'елки', 'жа', 'жен', 'зап', 'заруб', 'изд', 'иному', 'иностр', 'инст', 'канд', 'кв', 'кей', 'кис', 'корп', 'кэй', 'ля', 'мин', 'миним', 'мое', 'моему', 'моск', 'муж', 'нашему', 'нед', 'нить', 'палки', 'паф', 'пер', 'пиф', 'пос', 'пп', 'пр', 'просп', 'руб', 'своему', 'сек', 'спб', 'стр', 'твоему', 'тел', 'терр', 'тик', 'тов', 'тра', 'тс', 'тт', 'ту', 'тыс', 'ул', 'чегой', 'черт', 'чик', 'чтой', 'экз', 'яй'] not in stop_words.
  % sorted(inconsistent)


In [ ]:
lda_model = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=100, learning_method='batch', random_state = 2)
lda_res = lda_model.fit_transform(data_count_vectorized)

In [ ]:
# print function
all_topics = []
def print_topics(model, vectorizer, top_n=N_FIRST):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        all_topics.append([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

In [ ]:
print_topics(lda_model, count_vectorizer)

Topic 0:
[('книга', 1233.4009664821053), ('автор', 444.20915753077384), ('читать', 255.79388986254406), ('ещё', 198.41025537270352), ('большой', 157.55482810017782), ('написать', 149.95534262568214), ('есть', 135.07941274700696), ('текст', 129.05787385980912), ('год', 123.84183810125305), ('писать', 119.4501768025705), ('хороший', 117.06623557566667), ('литература', 113.3177090102682), ('страница', 110.54511762768333), ('информация', 106.92165036404516), ('читатель', 106.15691265668852), ('время', 103.81653207654138), ('прочитать', 101.24957437651747), ('человек', 97.12384421848157), ('факт', 93.91076700583248), ('история', 92.64287662109876)]
Topic 1:
[('человек', 697.9127026075483), ('жизнь', 358.61092202962016), ('есть', 211.63606399437967), ('время', 193.53482004447523), ('год', 184.043377469243), ('большой', 141.21765035117411), ('жить', 139.1980314836231), ('книга', 135.9179622877847), ('друг', 130.84404127445774), ('хороший', 130.83432238645233), ('хотеть', 125.21138858686652), 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[('детектив', 194.57709733871943), ('убийство', 80.9086779281285), ('дело', 70.93171348053096), ('расследование', 69.06666608102921), ('автор', 63.45613747700532), ('есть', 63.28792985531562), ('убийца', 61.97111915695696), ('роман', 58.05906477927098), ('детективный', 50.785042430561006), ('триллер', 46.16217465426825), ('персонаж', 45.91767926964421), ('жизнь', 43.490748798760954), ('человек', 41.2849320340133), ('линия', 41.238638028294396), ('хороший', 40.46155518706606), ('полицейский', 39.23096726982724), ('преступление', 38.64686182868552), ('действие', 36.779420268110556), ('книга', 31.316116672624908), ('мир', 30.761252839862856)]
Topic 8:
[('герой', 89.95748092415639), ('книга', 66.09222876453126), ('автор', 64.27880606252315), ('the', 62.066666485243786), ('персонаж', 40.742499329662415), ('рассказ', 39.46676323953834), ('год', 39.10686285918439), ('событие', 35.050722231561366), ('есть', 33.15783469990517), ('стать', 33.01359462417072), ('время', 31.859506843926432), ('сюже

In [ ]:
for i,topic in enumerate(all_topics):
    print("-------------------------------")
    print("Тема ",i)
    for pair in topic:
        print(pair[0])

-------------------------------
Тема  0
книга
автор
читать
ещё
большой
написать
есть
текст
год
писать
хороший
литература
страница
информация
читатель
время
прочитать
человек
факт
история
-------------------------------
Тема  1
человек
жизнь
есть
время
год
большой
жить
книга
друг
хороший
хотеть
роман
имя
стать
знать
мир
ещё
автор
война
сделать
-------------------------------
Тема  2
роман
автор
рассказ
два
часть
история
произведение
время
написать
читать
герой
прочитать
книга
мир
читатель
три
большой
повествование
век
писатель
-------------------------------
Тема  3
книга
герой
автор
сюжет
история
читать
главный
персонаж
ещё
героиня
конец
понравиться
два
большой
хороший
интересный
сказать
момент
neutral
время
-------------------------------
Тема  4
книга
герой
читать
прочитать
игра
фильм
рамка
хороший
человек
neutral
написать
понравиться
время
интересный
жизнь
страница
2020
хотеть
главный
читаться
-------------------------------
Тема  5
мир
год
человек
школа
магия
остров
герой
оказывать

In [ ]:
!pip install pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_count_vectorized, count_vectorizer, mds='tsne')
panel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:827: FutureWarning: 'square_distan

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3      114.344879  114.308296       1        1  30.931561
1      105.821045  -93.749626       2        1  16.662975
0     -138.334473   95.320763       3        1  13.722717
2       -7.436697  111.259697       4        1   5.880275
10    -207.550629 -105.472603       5        1   4.534972
4       52.817650   14.603765       6        1   4.266216
5      168.010849    6.806950       7        1   4.226208
7      -94.169601 -157.789551       8        1   3.878540
14    -122.849174  -39.059418       9        1   3.248452
8      -94.894264  195.408295      10        1   2.596499
9       24.927401 -180.464874      11        1   2.527751
12     -53.413467   26.832108      12        1   2.151978
11     -12.255850  -72.093201      13        1   2.035009
13    -224.217957   28.070581      14        1   1.885714
6       36.154255  209.427582      15        1   1.451134, topic_info=           Term         Freq        Total Category  logprob  loglift
609       книга  3900.000000  3900.000000  Default  30.0000  30.0000
421     женщина   440.000000   440.000000  Default  29.0000  29.0000
1450    ребёнок   425.000000   425.000000  Default  28.0000  28.0000
261       герой  1119.000000  1119.000000  Default  27.0000  27.0000
1925    человек  1312.000000  1312.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1939     читать    10.814430   956.615039  Topic15  -5.1622  -0.2497
310         два    10.292167   657.289624  Topic15  -5.2117   0.0761
883   несколько     9.224752   318.860946  Topic15  -5.3212   0.6900
260     героиня     9.317222   432.880659  Topic15  -5.3112   0.3942
261       герой     8.580449  1119.559199  Topic15  -5.3936  -0.6384

[943 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
5         1  0.113021    14
5         2  0.113021    14
5         7  0.678128    14
5        12  0.113021    14
7         1  0.113538    16
...     ...       ...   ...
1985      7  0.071364  язык
1985     10  0.017841  язык
1985     11  0.031222  язык
1985     12  0.129347  язык
1985     14  0.022301  язык

[3421 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 1, 3, 11, 5, 6, 8, 15, 9, 10, 13, 12, 14, 7])

LSA:

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
NUM_TOPICS = 20
N_FIRST = 20 # сколько первых слов на тему выводить
MIN_DF = 10 # минимальное число вхождений слова в документ, чтобы мы его учитывали (включали в вектор документа)
MAX_DF = 0.90 # в каком проценте документов должно присутствовать слово, чтобы оно не рассматривалось
STW_PATH = '/content/swl_optimum.txt' # путь к документу со стоп-словами

In [ ]:
lsa_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=5, random_state=None, tol=0.0)
lsa_model.fit_transform(data_count_vectorized)

/usr/local/lib/python3.7/dist-packages/scipy/linalg/decomp_qr.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  kwargs['lwork'] = ret[-2][0].real.astype(numpy.int)
/usr/local/lib/python3.7/dist-packages/scipy/linalg/decomp_qr.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, ch

array([[ 3.72479483, -2.14834764,  1.00868148, ...,  0.07893681,
         0.6997676 ,  0.74883613],
       [ 0.36518996,  0.05387677, -0.13789114, ...,  0.03183638,
         0.10730598, -0.17042986],
       [ 0.18292258,  0.06377532, -0.17531879, ...,  0.05067147,
         0.17597247, -0.13691174],
       ...,
       [ 0.83484645, -0.66922963,  0.35550452, ..., -0.02244468,
        -0.09729042, -0.04466986],
       [ 1.20703316,  0.91055238, -0.14639007, ..., -0.02816728,
         0.03503368, -1.24643042],
       [ 1.12384907, -0.31492364,  0.12656703, ...,  0.28338081,
        -0.13305974, -0.18143233]])

In [ ]:
# print function
all_topics = []
def print_topics(model, vectorizer, top_n=N_FIRST):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        all_topics.append([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

In [ ]:
print_topics(lsa_model, count_vectorizer)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topic 0:
[('книга', 0.5817176589594641), ('автор', 0.2531182684610932), ('человек', 0.2178710684537897), ('герой', 0.16367182310714457), ('ещё', 0.15742719001775618), ('жизнь', 0.13518708616202757), ('читать', 0.1301837260784458), ('время', 0.12975512451730217), ('история', 0.12716215177400106), ('год', 0.12392753755541418), ('роман', 0.12062935891637246), ('есть', 0.11804678898553253), ('большой', 0.11199364353579276), ('хороший', 0.1001360001935606), ('два', 0.09676745506174884), ('сюжет', 0.08973222876196776), ('женщина', 0.08940142325988093), ('главный', 0.08775152039343441), ('мир', 0.082413068188082), ('персонаж', 0.08014516092961951)]
Topic 1:
[('женщина', 0.4572855711707423), ('человек', 0.18114407804936453), ('мужчина', 0.17868904088430562), ('роман', 0.1615310375524478), ('время', 0.1375887360167377), ('жизнь', 0.13506148111272484), ('год', 0.11336657430860157), ('любовь', 0.11004226646192228), ('иметь', 0.0978382119496005), ('проститутка', 0.09738964735347391), ('есть', 0.09

In [ ]:
for i,topic in enumerate(all_topics):
    print("-------------------------------")
    print("Тема ",i)
    for pair in topic:
        print(pair[0])

-------------------------------
Тема  0
книга
автор
человек
герой
ещё
жизнь
читать
время
история
год
роман
есть
большой
хороший
два
сюжет
женщина
главный
мир
персонаж
-------------------------------
Тема  1
женщина
человек
мужчина
роман
время
жизнь
год
любовь
иметь
проститутка
есть
преступление
случай
мир
имя
ещё
сильный
дело
друг
говорить
-------------------------------
Тема  2
женщина
книга
мужчина
проститутка
преступление
иметь
случай
ребёнок
любовь
сильный
являться
часто
случайный
мужской
интеллект
чувство
выразить
признак
цель
инстинкт
-------------------------------
Тема  3
роман
герой
автор
сюжет
часть
женщина
главный
ещё
персонаж
героиня
два
читатель
три
мужчина
уэллс
страница
произведение
повествование
читать
история
-------------------------------
Тема  4
автор
сказка
тема
русский
факт
текст
военный
понимать
читатель
народ
материал
рассказ
литература
исследование
год
проститутка
война
страна
власть
система
-------------------------------
Тема  5
роман
человек
жизнь
часть
книг

In [ ]:
# визуализация
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
try:
  panel = pyLDAvis.sklearn.prepare(lsa_model, data_count_vectorized, count_vectorizer, mds='tsne')
except ValueError:
  pass
panel

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/sklearn.py:26: RuntimeWarning: invalid value encountered in true_divide
  return dists / dists.sum(axis=1)[:, None]


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3      114.344879  114.308296       1        1  30.931561
1      105.821045  -93.749626       2        1  16.662975
0     -138.334473   95.320763       3        1  13.722717
2       -7.436697  111.259697       4        1   5.880275
10    -207.550629 -105.472603       5        1   4.534972
4       52.817650   14.603765       6        1   4.266216
5      168.010849    6.806950       7        1   4.226208
7      -94.169601 -157.789551       8        1   3.878540
14    -122.849174  -39.059418       9        1   3.248452
8      -94.894264  195.408295      10        1   2.596499
9       24.927401 -180.464874      11        1   2.527751
12     -53.413467   26.832108      12        1   2.151978
11     -12.255850  -72.093201      13        1   2.035009
13    -224.217957   28.070581      14        1   1.885714
6       36.154255  209.427582      15        1   1.451134, topic_info=           Term         Freq        Total Category  logprob  loglift
609       книга  3900.000000  3900.000000  Default  30.0000  30.0000
421     женщина   440.000000   440.000000  Default  29.0000  29.0000
1450    ребёнок   425.000000   425.000000  Default  28.0000  28.0000
261       герой  1119.000000  1119.000000  Default  27.0000  27.0000
1925    человек  1312.000000  1312.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1939     читать    10.814430   956.615039  Topic15  -5.1622  -0.2497
310         два    10.292167   657.289624  Topic15  -5.2117   0.0761
883   несколько     9.224752   318.860946  Topic15  -5.3212   0.6900
260     героиня     9.317222   432.880659  Topic15  -5.3112   0.3942
261       герой     8.580449  1119.559199  Topic15  -5.3936  -0.6384

[943 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
5         1  0.113021    14
5         2  0.113021    14
5         7  0.678128    14
5        12  0.113021    14
7         1  0.113538    16
...     ...       ...   ...
1985      7  0.071364  язык
1985     10  0.017841  язык
1985     11  0.031222  язык
1985     12  0.129347  язык
1985     14  0.022301  язык

[3421 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 1, 3, 11, 5, 6, 8, 15, 9, 10, 13, 12, 14, 7])

NMF:

In [ ]:
from sklearn.decomposition import NMF

In [ ]:
NUM_TOPICS = 10
N_FIRST = 20 # сколько первых слов на тему выводить
MIN_DF = 5 # минимальное число вхождений слова в документ, чтобы мы его учитывали (включали в вектор документа)
MAX_DF = 0.90 # в каком проценте документов должно присутствовать слово, чтобы оно не рассматривалось
STW_PATH = '/content/swl_optimum.txt' # путь к документу со стоп-словами

In [ ]:
nmf_model = nmf = NMF(n_components=10, random_state=1, beta_loss="kullback-leibler", solver="mu", max_iter=1000, alpha=0.1, l1_ratio=0.5).fit(data_count_vectorized)
nmf_model.fit_transform(data_count_vectorized)

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1425: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/scipy/linalg/decomp_qr.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/re

array([[1.37023586e-01, 0.00000000e+00, 2.32895991e-04, ...,
        0.00000000e+00, 4.98271799e-02, 0.00000000e+00],
       [6.01062723e-14, 2.03369677e-21, 0.00000000e+00, ...,
        0.00000000e+00, 7.06446423e-24, 1.16520441e-20],
       [9.94976349e-03, 1.23911055e-31, 0.00000000e+00, ...,
        0.00000000e+00, 5.06296586e-03, 1.57503688e-02],
       ...,
       [2.34593222e-02, 0.00000000e+00, 1.93643739e-09, ...,
        3.03420257e-14, 0.00000000e+00, 2.75799349e-39],
       [5.18552107e-02, 7.35646887e-02, 0.00000000e+00, ...,
        2.07672873e-07, 4.58418840e-10, 1.70144577e-01],
       [8.19853402e-02, 0.00000000e+00, 3.04500844e-21, ...,
        0.00000000e+00, 0.00000000e+00, 1.47559858e-09]])

In [ ]:
# print function
all_topics = []
def print_topics(model, vectorizer, top_n=N_FIRST):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        all_topics.append([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

In [ ]:
print_topics(nmf_model, count_vectorizer)

Topic 0:
[('книга', 12.094291426299153), ('читать', 3.1405333570718237), ('ещё', 2.882579017512207), ('хороший', 2.3011393032208214), ('сюжет', 1.9004575623208617), ('автор', 1.741498889910753), ('два', 1.609975099375109), ('прочитать', 1.5808206985004034), ('написать', 1.4837404491544406), ('конец', 1.4815347762179336), ('есть', 1.4607686661539125), ('большой', 1.3195684537835397), ('герой', 1.2953413436264487), ('персонаж', 1.2852812148676893), ('страница', 1.2707735359183208), ('год', 1.1953406133848832), ('сказать', 1.1801330748492396), ('главный', 1.179807803911488), ('знать', 1.090402250621978), ('neutral', 1.0653141681984286)]
Topic 1:
[('ещё', 2.259234615391573), ('женщина', 1.967780734979697), ('есть', 1.8001893571030947), ('роман', 1.5696709557301598), ('знать', 1.3703471428785186), ('человек', 1.3501286751453985), ('хороший', 1.3143939928784611), ('хотеть', 1.1381605182184436), ('время', 1.1169651269305858), ('год', 1.086236101392856), ('имя', 1.0563207409588282), ('говорить

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[('герой', 6.513912151061693), ('главный', 2.429490978935736), ('автор', 2.2604461294204046), ('человек', 1.4017036905129474), ('жизнь', 0.8587935380068275), ('сюжет', 0.78409695117405), ('героиня', 0.7696086070394664), ('друг', 0.7683054270617438), ('чувство', 0.7644986923284599), ('персонаж', 0.7303659031264684), ('произведение', 0.598870642413772), ('пытаться', 0.563028774812707), ('хотеться', 0.5127037390989986), ('любить', 0.5054823325061544), ('думать', 0.4828603041967427), ('девушка', 0.48096209422924796), ('проблема', 0.47745322261670214), ('сторона', 0.4676573074891708), ('neutral', 0.4408746585935271), ('главное', 0.4339397417737952)]
Topic 7:
[('история', 4.685922653699042), ('понравиться', 1.8122348335050265), ('часть', 1.7571229093630643), ('рассказ', 1.5407026525555694), ('большой', 1.471259079094498), ('персонаж', 1.2551033658902468), ('neutral', 1.123082873564464), ('интересный', 1.047083995132395), ('сюжет', 1.0357209871024593), ('детектив', 1.0129243074475158), ('врем

In [ ]:
for i,topic in enumerate(all_topics):
    print("-------------------------------")
    print("Тема ",i)
    for pair in topic:
        print(pair[0])

-------------------------------
Тема  0
книга
читать
ещё
хороший
сюжет
автор
два
прочитать
написать
конец
есть
большой
герой
персонаж
страница
год
сказать
главный
знать
neutral
-------------------------------
Тема  1
ещё
женщина
есть
роман
знать
человек
хороший
хотеть
время
год
имя
говорить
мужчина
друг
любовь
большой
жизнь
дело
должный
понять
-------------------------------
Тема  2
книга
женщина
мужчина
любовь
ребёнок
время
иметь
neutral
преступление
тема
проститутка
случай
являться
девочка
узнать
сильный
девушка
дать
пример
женский
-------------------------------
Тема  3
человек
книга
год
жизнь
есть
время
стать
жить
знать
хотеть
конец
имя
большой
сказать
война
иметь
работа
остров
сделать
считать
-------------------------------
Тема  4
автор
текст
читатель
история
тема
есть
часть
факт
литература
большой
ещё
человек
информация
the
произведение
пример
русский
год
время
целое
-------------------------------
Тема  5
роман
жизнь
книга
произведение
человек
автор
два
часть
читать
время
напис

In [ ]:
# визуализация
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(nmf_model, data_count_vectorized, count_vectorizer, mds='tsne')
panel

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1425: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/pyLDAvis/sklearn.py:26: RuntimeWarning: invalid value encountered in true_divide
  return dists / dists.sum(axis=1)[:, None]


ValidationError: ignored